# 처음 배우는 셸 스크립트#7

## 3부 예제와 함께 활용하는 셸 스크립트 활용
시스템 운영, 개발환경 구축, 클라우드 시스템 운영 시 발생하는 다양한 상황에서 셸 스크립트를 활용한다.  
상황 -> 방법 찾기 -> 스크립트 생성 -> 문제 해결  

## 10 보안



In [ ]:
%%bash
#!/bin/bash
date
date -d yesterday #특정 일자의 시간을 확인하는 옵션
date '+%Y-%m-%d %l:%M %p'
date '+%Y-%m-%d %l:%M %p' -d yesterday